*** CURRENTLY DONE: for now... optimization of function will be done after


Publisher nodes: 
1. statemachine_pub
2. Reading_pub
3. Info_pub

Subscriber nodes:
1. statemachine_sub
2. Reading_sub
3. Info_sub

Topics:
1. Statemachine
2. Reading
3. Info


Reading is for final values/ read stuff
Info is for "initalizing, correct baud detected etc" such comments

Available states: 
1. "DISCONNECTED"
2. "CONNECTED"
3. "IDLE"
4. "SCANNING"


baudrate == 115200
node rate = 10

self.parameter1  #ACC
self.parameter2  #GYRO
self.parameter3  #Bandwidth
self.parameter4  #Sendback rate 
** parameters can only be entered a certain way in order to access the dict? this is done in a way such that they can choose from a range of parameters 

Structure: All parameters/ states are stored in the IMUCHECK class
Logic flow:
1. Initialize everything (nodes,topics,run,intialize etc)
2. Press button --> self.button changes and hence enables other if conditions
3. Once button pressed, code runs --> self.state changes 
4. 4 modes --> CONNECT, SCAN, SAVE, CLOSE
5. Each button press will end trigger service once






In [41]:
import time
import math
from pprint import PrettyPrinter
import rospy
#from smach import State, StateMachine
#import smach_ros
logger = rospy
import binascii

from serial import Serial
from std_msgs.msg import String
from IMUPublisher import publisher
from IMUSubscriber import subscriber
from IMUcontrol.srv import IMUcontrol


ModuleNotFoundError: No module named 'IMUcontrol'

In [28]:
# Settings 
# Refer to section 12.2 datasheet


NODE_RATE = 10
MOTOR_TOLERANCE = 0.0001
FIXED_BAUDRATE = 115200



WIT_IMU_CMD_CALI_ACC = b"\xff\xaa\x01\x01\x00" 
WIT_IMU_CMD_SAVE_CFG = b"\xff\xaa\x00\x00\x00"
WIT_IMU_CMD_UNLOCK = b"\xff\xaa\x69\x88\xb5"
WIT_IMU_CMD_SET_BAUDRATE = b"\xff\xaa\x04\x06\x00"  # 115200

ACC_SPEED = {
    "2": b"\xff\xaa\x21\x00\x00", #2g
    "4": b"\xff\xaa\x21\x01\x00", #4g
    "8": b"\xff\xaa\x21\x02\x00", #8g
    "16": b"\xff\xaa\x21\x03\x00", #16g
}

#double check GYRO DEGREES
GYRO_DEGREES = { 
    "250": b"\xff\xaa\x20\x00\x00",
    "500": b"\xff\xaa\x20\x01\x00",
    "1000": b"\xff\xaa\x20\x02\x00",
    "2000": b"\xff\xaa\x20\x03\x00", 
}


BANDWIDTH_HZ = {
    "200": b"\xff\xaa\x1f\x01\x00",
    "98" : b"\xff\xaa\x1f\x02\x00",
    "42": b"\xff\xaa\x1f\x03\x00",
    "20": b"\xff\xaa\x1f\x04\x00",
    "10": b"\xff\xaa\x1f\x05\x00",
    "5": b"\xff\xaa\x1f\x06\x00",
}


SENDBACK_RATE_HZ = {
    "0.1":b"\xff\xaa\x03\x01\x00", 
    "0.5": b"\xff\xaa\x03\x02\x00",
    "1": b"\xff\xaa\x03\x03\x00",
    "2": b"\xff\xaa\x03\x04\x00",
    "5": b"\xff\xaa\x03\x05\x00",
    "10":b"\xff\xaa\x03\x06\x00",
    "20":b"\xff\xaa\x03\x07\x00",
    "50":b"\xff\xaa\x03\x08\x00" ,
    "100":b"\xff\xaa\x03\x09\x00",
    "125":b"\xff\xaa\x03\x0a\x00",
    "200":b"\xff\xaa\x03\x0b\x00",
    "Once":b"\xff\xaa\x03\x0c\x00",
    "None":b"\xff\xaa\x03\x0d\x00",
}

WIT_IMU_CMD_SET_SENDBACK_CONTENT = b"\xff\xaa\x02\x0e\x02"
WIT_IMU_CMD_RESET_YAW = b"\xff\xaa\x01\x04\x00"

# convert 8bit hex to signed int
def c8h2sint(value):
    return -(value & 0x80) | (value & 0x7F)


def imu_feedback_parse(frame): #calculations equations given in datasheet
    [
        # convert linear acc
        _, _, _AxL, _AxH, _AyL, _AyH, _AzL, _AzH, _TL, _TH, _,
        # convert angular velocity
        _, _, _wxL, _wxH, _wyL, _wyH, _wzL, _wzH, _TL, _TH, _,
        # convert Roll Pitch Yaw
        _, _, _RollL, _RollH, _PitchL, _PitchH, _YawL, _YawH, _TL, _TH, _,
        # _,_,_,_,_,_,_,_,_,_,__,
        # convert quaternion
        _, _, _Q0L, _Q0H, _Q1L, _Q1H, _Q2L, _Q2H, _Q3L, _Q3H, _,
    ] = frame

    # m/s^2
    Ax = float((c8h2sint(_AxH) << 8) | _AxL) / 32768.0 * 16 * 9.8
    Ay = float((c8h2sint(_AyH) << 8) | _AyL) / 32768.0 * 16 * 9.8
    Az = float((c8h2sint(_AzH) << 8) | _AzL) / 32768.0 * 16 * 9.8

    # deg/s to rad/s
    wx = float((c8h2sint(_wxH) << 8) | _wxL) / 32768.0 * 2000 * math.pi / 180
    wy = float((c8h2sint(_wyH) << 8) | _wyL) / 32768.0 * 2000 * math.pi / 180
    wz = float((c8h2sint(_wzH) << 8) | _wzL) / 32768.0 * 2000 * math.pi / 180

    # deg/s to rad/s
    roll = float((c8h2sint(_RollH) << 8) | _RollL) / 32768.0 * 180 * math.pi / 180
    pitch = float((c8h2sint(_PitchH) << 8) | _PitchL) / 32768.0 * 180 * math.pi / 180
    yaw = float((c8h2sint(_YawH) << 8) | _YawL) / 32768.0 * 180 * math.pi / 180

    # quaternion from IMU
    q0 = float((c8h2sint(_Q0H) << 8) | _Q0L) / 32768
    q1 = float((c8h2sint(_Q1H) << 8) | _Q1L) / 32768
    q2 = float((c8h2sint(_Q2H) << 8) | _Q2L) / 32768
    q3 = float((c8h2sint(_Q3H) << 8) | _Q3L) / 32768

    return {
        "acceleration": [
            Ax,
            Ay,
            Az,
        ],
        "angle_speed": [
            wx,
            wy,
            wz,
        ],
        "radians": [
            roll,
            pitch,
            yaw,
        ],
        "quaternion": [q0, q1, q2, q3],
    }


In [29]:

#reading, info, state machine publisher nodes
'''
def publisher():
    global pub_statemachine_pub
    global pub_reading_pub
    global pub_info_pub
    pub_statemachine_pub = rospy.Publisher("StateMachine", String, queue_size=10)
    pub_reading_pub=rospy.Publisher("Reading",String,queue_size=10)
    pub_info_pub=rospy.Publisher("Info",String,queue_size=10)
    rospy.init_node("Publisher")
    rate = rospy.Rate(NODE_RATE)
'''




In [30]:
# reading, info, state machine subscriber nodes  
'''
def subscriber():
    rospy.init_node("Subscriber",anonymous=True)
    global sub_statemachine_sub
    sub_statemachine_sub = rospy.Subscriber("StateMachine",String,queue_size=10)
    global sub_reading_sub
    sub_reading_sub = rospy.Subscriber("Reading", String, queue_size=10)
    global sub_info_sub
    sub_info_sub = rospy.Subscriber("Info", String,queue_size=10)
'''




In [38]:

#Checking script
class IMUCHECK(object):
    def __init__(self):
        super(IMUCHECK,self).__init__()
        self.port = "/dev/ttyUSB0"
        self.baud = None
        self.button = None
        self.state = None
        self.parameter1 = ACC_SPEED[self.parameter1] #ACC
        self.parameter2 = GYRO_DEGREES[self.parameter2] #GYRO
        self.parameter3 = BANDWIDTH_HZ[self.parameter3] #Bandwidth
        self.parameter4 = SENDBACK_RATE_HZ[self.parameter4] #Sendback rate

    def initialize(self):
        self.state = "IDLE"


    def run(self):
        logger.logwarn("Running")
        pp = PrettyPrinter(indent=2)
        l = rospy.Rate(NODE_RATE)
  
        timeout =1.0
        while not rospy.is_shutdown:

            #CONNECT 
            if IMUCHECK.button == "CONNECT" and (IMUCHECK.state == "IDLE"):
                logger.logwarn("Trying to initialize....")
                logger.loginfo("Detecting IMU...")
                pub_info_pub.publish("CONNECTING")
                # TODO: Check if port exists and if parameters are accurate
                baud = FIXED_BAUDRATE
                logger.loginfo("Checking baudrate {}".format(baud))
                with Serial(port=self.port, baudrate=baud, timeout=1.0) as p:
                    timestart = time.time()
                    imu_raw_data = p.read_until(b"\x55\x51")
                    if time.time() - timestart < timeout: 
                        self.baud = baud
                        logger.logwarn("Baudrate detected {}".format(self.baud))
                        pub_info_pub.publish("Baudrate at {}".format(self.baud))
                        pub_info_pub.publish("Correct baudrate")
                        pub_statemachine_pub.publish("CONNECTED")
                        self.state = "CONNECTED"

                        #publish readings
                        for t in range(3):
                            frame_int = [int(x) for x in imu_raw_data]
                            frame_int = frame_int[-2:] + frame_int[:-2]
                            result = imu_feedback_parse(frame_int)
                            logger.loginfo("Got: \n {}".format(pp.pformat(result)))
                            pub_reading_pub.publish("Got: \n {}".format(pp.pformat(result)))
                    else:
                        logger.logwarn ("ERROR, not at correct baudrate")
                        pub_reading_pub.publish("Wrong baudrate, press SCAN to set")
                        pub_statemachine_pub.publish("IDLE")
                                    
            #SCAN
            elif self.button == "SCAN" and (self.state == 'CONNECTED' or self.state =="IDLE"):
                
                for baud in (115200,9600):
                    logger.loginfo("Checking baudrate {}".format(baud))
                    if self.baud is None:
                        logger.logwarn("No baudrate, please check")
                        pub_reading_pub.publish("No baudrate detected, please check")
                    with Serial(port=self.port, baudrate=baud, timeout=1.0) as p:
                        timestart = time.time()
                        imu_raw_data = p.read_until(b"\x55\x51")
                        if time.time() - timestart < timeout: 
                            self.baud = baud
                            logger.logwarn("Baudrate detected {}".format(self.baud))
                            pub_reading_pub.publish("Baudrate at {}".format(self.baud))
                            pub_statemachine_pub.publish("CONNECTED")
                            self.state = "CONNECTED"
                    
                        p.write(WIT_IMU_CMD_UNLOCK)
                        p.write(WIT_IMU_CMD_UNLOCK)
                        p.write(WIT_IMU_CMD_UNLOCK) 
                        p.write(WIT_IMU_CMD_UNLOCK)
                        time.sleep(0.2)
                        logger.info("baudrate is set to {}".format(self.baud))    
                        pub_info_pub.publish("baudrate is set to {}".format(self.baud))
                        p.write(WIT_IMU_CMD_SET_BAUDRATE)
                        self.baud = 115200
                        time.sleep(0.2)
                        logger.logwarn("Setting ACC scale to: 16g/s2!")
                        pub_info_pub.publish("Setting ACC scale to: 16g/s2!")
                        p.write(self.parameter1) #WIT_IMU_CMD_SET_SCALE_ACC
                        time.sleep(0.2)
                        logger.logwarn("Setting gyro scale to: 2000deg/s!")
                        pub_info_pub.publish("Setting gyro scale to: 2000deg/s!")
                        p.write(self.parameter2) #WIT_IMU_CMD_SET_SCALE_GYRO
                        time.sleep(0.2)
                        logger.logwarn("Setting bandwidth to: 250Hz!")
                        pub_info_pub.publish("Setting bandwidth to: 250Hz!")
                        p.write(self.parameter3) #WIT_IMU_CMD_SET_BANDWIDTH
                        time.sleep(0.2)
                        logger.logwarn("Setting feedback rate to: 200Hz!")
                        pub_info_pub.publish("Setting feedback rate to: 200Hz!")
                        p.write(self.parameter4) #WIT_IMU_CMD_SET_SENDBACK_RATE)
                        time.sleep(0.2)
                        logger.logwarn(
                        "Setting feedback content as:\n0x51->acc 0x52->w 0x53->angle 0x59->quaternion"
                        )
                        p.write(WIT_IMU_CMD_SET_SENDBACK_CONTENT)
                        time.sleep(0.2)
                        
                        #publish readings
                        for t in range(3):
                            frame_int = [int(x) for x in imu_raw_data]
                            frame_int = frame_int[-2:] + frame_int[:-2]
                            result = imu_feedback_parse(frame_int)
                            logger.loginfo("Got: \n {}".format(pp.pformat(result)))
                            pub_reading_pub.publish("Got: \n {}".format(pp.pformat(result)))

                #SAVE
            elif (self.button =="SAVE") and (self.state == "CONNECTED"):
                        p.write(WIT_IMU_CMD_SAVE_CFG)
                        logger.logwarn("CFG saved!")
                        pub_info_pub("CFG saved!")

                #CLOSE
            elif self.button == 'CLOSE' and self.state != "DISCONNECTED":
                logger.logwarn("Exiting")
                pub_info_pub("Exiting")
                pub_statemachine_pub.publish("DISCONNECTED")
                self.state == "DISCONNECTED"
                break

                        
        

In [33]:
#service node (backend call service node) 
def service_node():
    rospy.init_node("Service", anonymous=True)
    service_call = rospy.service("service_call", IMUcontrol,handle_imu_requests)
    rospy.loginfo("Service is ready")

    def handle_imu_requests(req):
        print("IMU CONNECTED")
        print(req.parameter1)
        print(req.parameter2)
        print(req.parameter3)
        print(req.parameter4)
        print(req.button)
        '''
        IMUCHECK.button = req.button
        IMUCHECK.parameter1 = req.parameter1
        IMUCHECK.parameter2 = req.parameter2
        IMUCHECK.parameter3 = req.parameter3
        IMUCHECK.parameter4 = req.paratmeter4
        IMUCHECK.initialize()
        IMUCHECK.run()
    '''



if __name__ == "__main__":
    publisher()
    subscriber()
    service_node()
    

ROSException: rospy.init_node() has already been called with different arguments: ('Statemachine_pub', ['/home/zhang/.local/lib/python3.8/site-packages/ipykernel_launcher.py', '--f=/home/zhang/.local/share/jupyter/runtime/kernel-v2-48870MKdwx2xaF9hi.json'], True, None, False, False)